<a href="https://colab.research.google.com/github/maychr/NLP_AES_promt-based/blob/main/Prompt_essay_classifications_with_transformer_models/zero-shot%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation of the Environment
## Colab

In [1]:
# Installation of the following additional packages
!pip install -q git+https://github.com/huggingface/transformers.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 4.2 MB/s 
     |████████████████████████████████| 6.5 MB 68.8 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 


# Data Import


In [ ]:
# Import test dataset from GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("//content/gdrive/MyDrive/nlp_data/") 

import numpy
import pandas as pd
data = pd.read_csv("test_recoded_final.csv", sep= ";", encoding="UTF-8")


In [3]:
# Casting the data into lists
label = data.iloc[0:, 1].to_list()
mail = data.iloc[0:, 0].to_list()


# Zero-Shot 
## Prepare and run the Zero-Shot-Classification Pipepline

In [4]:
# Load pipeline for zero-shot-classification including model and matching tokenizer
# Define and load the pre-trained model that will be used
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [5]:
# Run the pipeline
runs = len(mail)
sequence_to_classify = mail[0:runs]

# Define labels for zero-shot classification
candidate_labels = ['unhöflich', 'höflich']

# Predict labels
prediction = classifier(sequence_to_classify, candidate_labels)


In [6]:
# Check predictions
label = prediction

# Print output example
prediction[0]


{'labels': ['höflich', 'unhöflich'],
 'scores': [0.7901485562324524, 0.2098514586687088],
 'sequence': 'Hall Frau Meier,\n\nangehängt finden sie die vervollständigte Liste zur Abweichungsanalyse. DIe auffälligen Abweichungen habe ich in Orange makiert, ausgewählt habe ich diese nicht nur nach der absoluten Höhe des Betrages, sondern nur, wenn die Abweichung für diesen Kunden über 20% von den Plan'}

## Further processing of model output

In [7]:
# Extract labels from prediction
label = [ele['labels'] for ele in label]


In [8]:
label = [ele[0] for ele in label]


In [9]:
pred_class = label


In [10]:
# Replace labels with 0 and 1 for evaulation
import pandas as pd
test_pred_class = pd.DataFrame(data=pred_class)

test_pred_class = test_pred_class.replace({'höflich':1, 'unhöflich':0})


In [11]:
# Get human labels for evaluation
test_label = data.iloc[:,1]


# Calculate Evaluation Metrics

In [12]:
 # Checking the test data results by comparing human gold-labels and model prediction
from sklearn import metrics

test_label = test_label[0:runs]

# Mean accuracy
print("Mean Accuracy:\n", metrics.accuracy_score(test_label, test_pred_class))

# Confusion matrix
print("Confusion Matrix:\n", metrics.confusion_matrix(test_label, test_pred_class))

# F1 Score
print("F1 Score:\n", metrics.f1_score(test_label, test_pred_class))

# Precision
print("Precision:\n", metrics.precision_score(test_label, test_pred_class))

# Recall
print("Recall:\n", metrics.recall_score(test_label, test_pred_class))

# ROC AUC Score
print("ROC AUC:\n", metrics.roc_auc_score(test_label, test_pred_class))

# Cohen's Kappa Score
print("Cohen's Kappa:\n", metrics.cohen_kappa_score(test_label, test_pred_class))


Mean Accuracy:
 0.7990430622009569
Confusion Matrix:
 [[ 22  28]
 [ 98 479]]
F1 Score:
 0.8837638376383764
Precision:
 0.9447731755424064
Recall:
 0.830155979202773
ROC AUC:
 0.6350779896013865
Cohen's Kappa:
 0.1647954329210276
